# Práctica 3: Representación del conocimiento

__Fecha de entrega: 8 de mayo de 2022__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 15__

__Nombres de los estudiantes: Javier Sande Ríos y Mario Sanz Guerrero__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [1]:
# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [2]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,es". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[Isaac Asimov (Q34981)](https://www.wikidata.org/wiki/Q34981) fue un escritor y profesor de bioquímica en la Universidad de Boston conocido por ser un prolífico autor de obras de ciencia ficción, historia y divulgación científica. 

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

#### Solución:

In [3]:
%%wdsparql
SELECT ?fecha ?lugarLabel ?lugar
WHERE 
{
  wd:Q34981 wdt:P569 ?fecha;
            wdt:P19 ?lugar.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,fecha,lugarLabel,lugar
0,1920-01-02T00:00:00Z,Petrovichi,http://www.wikidata.org/entity/Q1010621


Esta es una consulta muy sencilla. Para Asimov, cogemos su fecha y lugar de nacimiento.

### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

#### Solución:

In [4]:
%%wdsparql
SELECT ?profesionLabel
WHERE 
{
  wd:Q34981 wdt:P106 ?profesion.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?profesionLabel

,profesionLabel
0,autobiographer
1,biochemist
2,journalist
3,non-fiction writer
4,novelist
5,prosaist
6,science fiction writer
7,science writer
8,scientist
9,screenwriter


Vuelve a ser una consulta muy sencilla. Para Asimov, cogemos todas sus profesiones y las ordenamos alfabéticamente con `ORDER BY`.

### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

#### Solución:

In [5]:
%%wdsparql
SELECT ?profesionLabel
WHERE 
{
  wd:Q34981 wdt:P106 ?profesion.
  ?profesion wdt:P279* wd:Q36180.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?profesionLabel

,profesionLabel
0,autobiographer
1,biochemist
2,journalist
3,non-fiction writer
4,novelist
5,prosaist
6,science fiction writer
7,science writer
8,scientist
9,screenwriter


Para Asimov, cogemos sus profesiones que son subclases de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180) y las ordenamos alfabéticamente con `ORDER BY`.

Nos llama la atención que se devuelvan todas las 12 profesiones que se han devuelto para la consulta anterior. Hay algunas profesiones como _autobiographer_ que es obvio que son subclase de _escritor_, pero otras que no lo son tanto como por ejemplo _biochemist_. Por ese motivo, vamos a meternos en la web de Wikidata para ver a qué nivel son subclase de _escritor_ estas profesiones que no parecen tan relacionadas (la flecha hacia la derecha representa "es subclase de"):

* Biochemist: _biochemist_ &#8594; _biologist_ &#8594; _scientist_ &#8594; _researcher_ &#8594; _non-fiction writer_ &#8594; _writer_
* Scientist: _scientist_ &#8594; _researcher_ &#8594; _non-fiction writer_ &#8594; _writer_
* University teacher: _university teacher_ &#8594; _academic_ &#8594; _scholar_ &#8594; _researcher_ &#8594; _non-fiction writer_ &#8594; _writer_

Las demás profesiones parecen bastante más relacionadas con _escritor_. Esto ha servido para demostrar que la consulta está bien hecha.

### Consulta 4

Asimov se casó más de una vez. Para cada uno de los matrimonios queremos conocer el nombre de la esposa y las fechas de inicio y finalización. Los resultados deben aparecer ordenados cronológicamente.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

#### Solución:

In [6]:
%%wdsparql
SELECT ?esposa ?esposaLabel ?inicio ?finalizacion
WHERE
{
  ?esposa wdt:P26 wd:Q34981.
  ?esposa p:P26 [ pq:P580 ?inicio;
                  pq:P582 ?finalizacion ].
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?inicio)

,esposa,esposaLabel,inicio,finalizacion
0,http://www.wikidata.org/entity/Q107693884,Gertrude Asimov,1942-07-26T00:00:00Z,1973-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q517435,Janet Asimov,1973-01-01T00:00:00Z,1992-01-01T00:00:00Z


Aquí ya tenemos consultas algo más complejas, así que creemos conveniente explicar lo que hace la consulta.

En primer lugar, busca todas las personas cuyo cónyugue es (o fue) Isaac Asimov. Para cada una de estas personas, mira en su categoría de matrimonio y guarda la fecha de inicio y finalización del matrimonio. Esta segunda búsqueda se hace mediante el uso de calificadores de SPARQL. La `p:` indica el sujeto de la nueva tripleta que vamos a consultar, y la `pq:` es el calificativo que queremos mirar, que en este caso son la fecha de inicio y fin. Finalmente, se ordenan cronológicamente los resultados recuperados por la consulta con `ORDER BY`.

### Consulta 5

Asimov recibió muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha y la obra por la que fue premiado (si están disponibles). Los resultados se deben mostrar ordenador por fecha.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

#### Solución:

In [7]:
%%wdsparql
SELECT ?premio ?premioLabel ?obra ?obraLabel ?fecha
WHERE
{
  wd:Q34981 p:P166 ?statement . # statement: necesario para acceder a los qualifiers
  ?statement ps:P166 ?premio {
             ?statement pq:P585 ?fecha
             OPTIONAL {?statement pq:P1686 ?obra}
    }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?fecha)

,premio,premioLabel,obra,obraLabel,fecha
0,http://www.wikidata.org/entity/Q188914,Hugo Award,None,None,1963-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q6143927,James T. Grady-James H. Stack Award for Interp...,None,None,1965-01-01T00:00:00Z
2,http://www.wikidata.org/entity/Q188914,Hugo Award,http://www.wikidata.org/entity/Q20899148,Foundation Trilogy,1966-01-01T00:00:00Z
3,http://www.wikidata.org/entity/Q987744,Edward E. Smith Memorial Award,None,None,1967-01-01T00:00:00Z
4,http://www.wikidata.org/entity/Q266012,Nebula Award for Best Novel,http://www.wikidata.org/entity/Q389750,The Gods Themselves,1972-01-01T00:00:00Z
5,http://www.wikidata.org/entity/Q255032,Hugo Award for Best Novel,http://www.wikidata.org/entity/Q389750,The Gods Themselves,1973-01-01T00:00:00Z
6,http://www.wikidata.org/entity/Q595998,Locus Award for Best Novel,http://www.wikidata.org/entity/Q389750,The Gods Themselves,1973-01-01T00:00:00Z
7,http://www.wikidata.org/entity/Q3174253,Klumpke-Roberts Award,None,None,1975-01-01T00:00:00Z
8,http://www.wikidata.org/entity/Q936683,Nebula Award for Best Novelette,http://www.wikidata.org/entity/Q468132,The Bicentennial Man,1976-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q1056265,Hugo Award for Best Novelette,http://www.wikidata.org/entity/Q468132,The Bicentennial Man,1977-01-01T00:00:00Z


Para Isaac Asimov (`wd:Q34981`) definimos el statement de _award received_ (`p:P166`), y para cada uno de los premios recibidos accedemos a los qualifiers de la fecha en la que se recibió el premio (`pq:P585`) y la obra por la que se recibió el premio (`pq:P1686`). Este último qualifier es opcional porque puede estar vacío, que significa que esa información no está disponible. En el resultado devuelto por la consulta, esto se representa mediante la palabra '_None_'. Por último, ordenamos los resultados por fecha con `ORDER BY`.

### Consulta 6

Ahora queremos conocer todas las obras escritas por Asimov del tipo [obras literarias de ciencia ficción (Q3238422)](https://www.wikidata.org/wiki/Q3238422) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

#### Solución:

In [8]:
%%wdsparql
SELECT ?obra ?obraLabel ?genero ?generoLabel
WHERE
{
    ?obra wdt:P50 wd:Q34981;
          wdt:P136 ?genero .
    ?genero wdt:P279* wd:Q3238422 .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?obraLabel)

,obra,obraLabel,genero,generoLabel
0,http://www.wikidata.org/entity/Q603705,. . . That Thou Art Mindful of Him,http://www.wikidata.org/entity/Q21905924,science fiction short story
1,http://www.wikidata.org/entity/Q4631777,2430 A. D.,http://www.wikidata.org/entity/Q21905924,science fiction short story
2,http://www.wikidata.org/entity/Q4657902,A Loint of Paw,http://www.wikidata.org/entity/Q21905924,science fiction short story
3,http://www.wikidata.org/entity/Q3549799,A Statue for Father,http://www.wikidata.org/entity/Q21905924,science fiction short story
4,http://www.wikidata.org/entity/Q3618113,Anniversary,http://www.wikidata.org/entity/Q21905924,science fiction short story
5,http://www.wikidata.org/entity/Q2872379,Author! Author!,http://www.wikidata.org/entity/Q21905924,science fiction short story
6,http://www.wikidata.org/entity/Q4920795,Black Friar of the Flame,http://www.wikidata.org/entity/Q21905924,science fiction short story
7,http://www.wikidata.org/entity/Q4925104,Blank!,http://www.wikidata.org/entity/Q21905924,science fiction short story
8,http://www.wikidata.org/entity/Q3511886,Blind Alley,http://www.wikidata.org/entity/Q21905924,science fiction short story
9,http://www.wikidata.org/entity/Q2855170,Breeds There a Man...?,http://www.wikidata.org/entity/Q21905924,science fiction short story


Cogemos todas las obras de Isaac Asimov y nos guardamos su género para después quedarnos solo con aquellas obras cuyo género sea subgénero de [obras literarias de ciencia ficción (Q3238422)](https://www.wikidata.org/wiki/Q3238422). Finalmente, ordenamos las obras por orden alfabético con `ORDER BY`.

### Consulta 7

Vamos a volver a recuperar las mismas obras de la consulta anterior pero mostrando también la fecha de publicación de cada una. Las obras deben aparece aunque no tengan fecha de publicación asociada. En caso de que una obra tenga varias fechas de publicación, se mostrará sólo la más antigua. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por obra y aplicar una función de agregación sobre las fechas de publicación. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

#### Solución:

In [9]:
%%wdsparql
SELECT ?obra ?obraLabel ?genero ?generoLabel (MIN(?fecha) AS ?fechaMasAntigua)
WHERE
{
    ?obra wdt:P50 wd:Q34981;
          wdt:P136 ?genero;
    OPTIONAL{?obra wdt:P577 ?fecha} .
    ?genero wdt:P279* wd:Q3238422 .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?obra ?obraLabel ?genero ?generoLabel
ORDER BY ASC(?obraLabel)

,obra,obraLabel,genero,generoLabel,fechaMasAntigua
0,http://www.wikidata.org/entity/Q603705,. . . That Thou Art Mindful of Him,http://www.wikidata.org/entity/Q21905924,science fiction short story,1974-05-01T00:00:00Z
1,http://www.wikidata.org/entity/Q4631777,2430 A. D.,http://www.wikidata.org/entity/Q21905924,science fiction short story,1970-01-01T00:00:00Z
2,http://www.wikidata.org/entity/Q4657902,A Loint of Paw,http://www.wikidata.org/entity/Q21905924,science fiction short story,1957-01-01T00:00:00Z
3,http://www.wikidata.org/entity/Q3549799,A Statue for Father,http://www.wikidata.org/entity/Q21905924,science fiction short story,1959-01-01T00:00:00Z
4,http://www.wikidata.org/entity/Q3618113,Anniversary,http://www.wikidata.org/entity/Q21905924,science fiction short story,1959-03-01T00:00:00Z
5,http://www.wikidata.org/entity/Q2872379,Author! Author!,http://www.wikidata.org/entity/Q21905924,science fiction short story,1964-01-01T00:00:00Z
6,http://www.wikidata.org/entity/Q4920795,Black Friar of the Flame,http://www.wikidata.org/entity/Q21905924,science fiction short story,1942-01-01T00:00:00Z
7,http://www.wikidata.org/entity/Q4925104,Blank!,http://www.wikidata.org/entity/Q21905924,science fiction short story,1957-01-01T00:00:00Z
8,http://www.wikidata.org/entity/Q3511886,Blind Alley,http://www.wikidata.org/entity/Q21905924,science fiction short story,1945-03-01T00:00:00Z
9,http://www.wikidata.org/entity/Q2855170,Breeds There a Man...?,http://www.wikidata.org/entity/Q21905924,science fiction short story,1951-01-01T00:00:00Z


En esta consulta vemos pequeñas variaciones de resultados cuando metemos o quitamos restricciones. Vamos a ver una lista de estas restricciones que comentamos y el número de respuestas obtenidas:

| **Consulta** | **Contenido de la consulta** | **Nº de resultados** |
|---|---|---|
| 7.1 | Obras de Asimov | 416 |
| 7.2 | Obras de Asimov cuyo género es subgénero de<br>`literatura de ciencia ficcion`  | 125 |
| 7.3 | Obras de Asimov cuyo género es subgénero de <br>`literatura de ciencia ficcion` con fechas | 132 |
| 7.4 | Obras de Asimov cuyo género es subgénero de <br>`literatura de ciencia ficcion` solo cogiendo la última fecha | 125 |

Vamos a ir paso a paso explicando y analizando el por qué de estos resultados.

De la consulta 7.1 a la 7.2 se reduce mucho porque estamos pidiendo que solo nos devuelva las que tengan un género que sea subgénero de `literatura de ciencia ficcion`. Por ello, se descartan todas las obras de Asimov que no cumplen esto. Aquí cabe destacar que muchas de las obras que se devolvían en la consulta 7.1 estaban repetidas dos veces, una de ellas teniendo género `science fiction` y la otra otro género distinto. Por poner un ejemplo concreto, la obra [_"For the Birds"_](https://www.wikidata.org/wiki/Q5467012) tiene como géneros `science fiction` y `science fiction short story`, y esta última es subclase de `literatura de ciencia ficción`, así que en la primera consulta aparecía dos veces pero en la segunda consulta solo nos quedamos con la segunda aparición de esta obra en la primera consulta.

De la consulta 7.2 a la 7.3 vemos que hay un incremento de 7 obras devueltas. Esto se debe a que de las 132 obras devueltas, hay 7 que tienen más de una fecha de publicación, de tal manera que al pedir también la fecha en la consulta, la obra se ha duplicado para poder devolver las dos fechas de publicación. Concretamente, las obras en las que hay dos fechas de publicación distintas son [_"A Statue for Father"_](http://www.wikidata.org/entity/Q3549799), [_"Buy Jupiter"_](http://www.wikidata.org/entity/Q2962693), [_"Forward the Foundation"_](http://www.wikidata.org/entity/Q938629), [_"Prelude to Foundation"_](http://www.wikidata.org/entity/Q783533), [_"Rain, Rain, Go Away"_](http://www.wikidata.org/entity/Q3392605), y [_"The Ugly Little Boy"_](http://www.wikidata.org/entity/Q21190269), que esta última tiene 3 fechas de publicación distintas, sumando un total de 7 obras repetidas en la consulta 7.3 respecto a la 7.2 por fechas de publicación múltiples.

En la consulta 7.4, que corresponde a la celda de código de arriba porque es la solución al ejercicio, vemos que el valor vuelve a ser 125, que es justamente el comportamiento esperado al agrupar las obras por título y género y quedarnos con la fecha de publicación más antigua de todas ellas. Esto hace que nos quedemos solo con un resultado de las obras que tenían fechas de publicación múltiples, por lo que el resultado es el mismo que en la consulta 7.2, en la que no considerábamos fechas distintas.

Un comentario importante sobre esta última consulta es que hemos tenido que agrupar por título y género porque si no nos da `Bad aggregate`. Esto pasa porque para esta combinación de variables solo hay un resultado posible, así que podemos agruparlas todas juntas y quedarnos con la que tenga la fecha mínima, que será la más antigua.

Dejamos a continuación el código de estas consultas que se han hecho por si se quisieran probar (sin ejecutar para que no ocupen mucho espacio), siendo la consulta 7.4 la solución al ejercicio, por lo que está en la celda de código de arriba:

**Consulta 7.1:** Obras cuyo autor es Asimov

In [ ]:
%%wdsparql
SELECT ?obra ?obraLabel
WHERE
{
    ?obra wdt:P50 wd:Q34981;
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?obraLabel)

**Consulta 7.2:** Obras de Asimov cuyo género es subgénero de `literatura de ciencia ficción`

In [ ]:
%%wdsparql
SELECT ?obra ?obraLabel ?genero ?generoLabel
WHERE
{
    ?obra wdt:P50 wd:Q34981;
          wdt:P136 ?genero .
    ?genero wdt:P279* wd:Q3238422 .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?obraLabel)

**Consulta 7.3:** Obras de Asimov cuyo género es subgénero de `literatura de ciencia ficción` con fechas

In [ ]:
%%wdsparql
SELECT ?obra ?obraLabel ?genero ?generoLabel ?fecha
WHERE
{
    ?obra wdt:P50 wd:Q34981;
          wdt:P136 ?genero;
    OPTIONAL{?obra wdt:P577 ?fecha} .
    ?genero wdt:P279* wd:Q3238422 .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?obraLabel)

### Consulta 8

Asimov es sin duda un autor prolífico. ¿Cuántas obras escribió a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

#### Solución:

In [10]:
%%wdsparql
SELECT (COUNT(*) AS ?numero_de_obras)
WHERE 
{
  ?libro wdt:P50 wd:Q34981;
         wdt:P31 wd:Q7725634.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,numero_de_obras
0,272


La forma de devolver el número de obras escritas en la vida de Asimov que sean instancias directas de [literary work](https://www.wikidata.org/wiki/Q7725634) es haciendo la consulta de todas esas obras y contar los resultados devueltos con la funcion `COUNT(*)` de SPARQL.

### Consulta 9

De todas las obras que escribió a lo largo de su vida, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

#### Solución:

In [11]:
%%wdsparql
SELECT ?obra ?obraLabel ?fecha
WHERE
{
    ?obra wdt:P50 wd:Q34981;
          wdt:P577 ?fecha .
    FILTER(YEAR(?fecha) >= 1970 && YEAR(?fecha) <= 1980).
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?fecha)

,obra,obraLabel,fecha
0,http://www.wikidata.org/entity/Q4009001,Waterclap,1970-01-01T00:00:00Z
1,http://www.wikidata.org/entity/Q4631777,2430 A. D.,1970-01-01T00:00:00Z
2,http://www.wikidata.org/entity/Q4806996,Asimov's Guide to Shakespeare,1970-01-01T00:00:00Z
3,http://www.wikidata.org/entity/Q7765145,The Solar System and Back,1970-01-19T00:00:00Z
4,http://www.wikidata.org/entity/Q28808642,As Chemist to Chemist,1970-05-01T00:00:00Z
5,http://www.wikidata.org/entity/Q16932955,The Stars in their Courses,1971-01-01T00:00:00Z
6,http://www.wikidata.org/entity/Q389750,The Gods Themselves,1972-01-01T00:00:00Z
7,http://www.wikidata.org/entity/Q831556,Mirror Image,1972-01-01T00:00:00Z
8,http://www.wikidata.org/entity/Q2277195,The Early Asimov,1972-01-01T00:00:00Z
9,http://www.wikidata.org/entity/Q3345921,Q3345921,1972-01-01T00:00:00Z


En esta consulta obtenemos todas las obras escritas por Isaac Asimov y guardamos su fecha para mostrar solo aquellas obras en las cuales alguna fecha de publicación está entre 1970 y 1980. Esto de que se muestre si **alguna** fecha de publicación está en el rango que queremos ocurre porque si una obra tiene varias fechas de publicación se devuelve como resultados distintos, y solo la cogemos si alguno de estos resultados está en nuestro rango de años.

### Consulta 10

¿Qué otros escritores de ciencia ficción estudiaron en algún centro donde también estudió Asimov? Para cada uno de ellos muestra su nombre y fechas de nacimiento y defunción (si están disponibles) y el centro donde estudió. Muestra los resultados ordenados alfabéticamente por centro y luego por autor.

#### Solución:

In [12]:
%%wdsparql
SELECT ?persona ?personaLabel ?institucionLabel ?nacimiento ?defuncion
WHERE
{
  wd:Q34981 wdt:P69 ?institucion .
  ?persona wdt:P69 ?institucion ;
           wdt:P106 wd:Q18844224.
  OPTIONAL{ ?persona wdt:P569 ?nacimiento }.
  OPTIONAL{ ?persona wdt:P570 ?defuncion }.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?institucion ?personaLabel

,persona,personaLabel,institucionLabel,nacimiento,defuncion
0,http://www.wikidata.org/entity/Q34981,Isaac Asimov,Boys and Girls High School,1920-01-02T00:00:00Z,1992-04-06T00:00:00Z
1,http://www.wikidata.org/entity/Q34981,Isaac Asimov,Fu Foundation School of Engineering and Applie...,1920-01-02T00:00:00Z,1992-04-06T00:00:00Z
2,http://www.wikidata.org/entity/Q34981,Isaac Asimov,Columbia University School of General Studies,1920-01-02T00:00:00Z,1992-04-06T00:00:00Z
3,http://www.wikidata.org/entity/Q13563168,Alaya Dawn Johnson,Columbia University,1982-03-31T00:00:00Z,None
4,http://www.wikidata.org/entity/Q547414,Algis Budrys,Columbia University,1931-01-09T00:00:00Z,2008-06-09T00:00:00Z
5,http://www.wikidata.org/entity/Q4800570,Arthur W. Saha,Columbia University,1923-10-31T00:00:00Z,1999-11-19T00:00:00Z
6,http://www.wikidata.org/entity/Q4886274,Ben Parris,Columbia University,1961-01-01T00:00:00Z,None
7,http://www.wikidata.org/entity/Q5233934,David G. Hartwell,Columbia University,1941-07-10T00:00:00Z,2016-01-20T00:00:00Z
8,http://www.wikidata.org/entity/Q551929,David Kyle,Columbia University,1919-02-14T00:00:00Z,2016-09-18T00:00:00Z
9,http://www.wikidata.org/entity/Q5294012,Donald Barr,Columbia University,1921-08-08T00:00:00Z,2004-02-05T00:00:00Z


En esta consulta lo que hacemos es recuperar todas las instituciones donde estudió Isaac Asimov. Buscamos todas las personas que también estudiaron en cada una de esas instituciones y cuya ocupación es la de [escritor de ciencia ficción (Q18844224)](https://www.wikidata.org/wiki/Q18844224). Para todas esas personas, también recuperamos sus fechas de nacimiento y defunción en caso de estar disponibles, de ahí que sean tripletas opcionales. Finalmente, ordenamos alfabéticamente los resultados recuperados, primero por nombre de institución y luego por autor.

En este caso no necesitamos una tripleta que aserte que `?persona` es instancia de Humano (`Q5`) porque el mero hecho de haber sido educado en una institución (`P69`) le convierte en humano. Sin embargo, no estaría mal meter la tripleta `?persona wdt:P31 wd:Q5 .` en todas las consultas, pero nos daría el mismo resultado que sin ella.

Llama la atención en esta consulta que todas las personas recuperadas coinciden con Asimov solo en la Universidad de Columbia. Por si hubiera algún error en la consulta, vamos a realizar pruebas.

Asumiendo que la Universidad de Columbia se recupera bien, vamos a pasarla por alto y a centrarnos en las demás instituciones a las que acudió Asimov.

**Nota:** Las celdas de código (consultas) adicionales que creamos nosotros y que recuperan mucha información como la siguiente, las vamos a dejar sin ejecutar para que no se cree un documento gigantesco. En lugar de eso, comentaremos debajo el número de resultados recuperados (esto también podría hacerse con un `COUNT` como en la consulta 8, pero preferimos dejarlo así por si en un futuro se quisiera ejecutar para ver los resultados).

Estudiantes de [Fu Foundation School of Engineering and Applied Science (Q5149833)](https://www.wikidata.org/wiki/Q5149833):

In [ ]:
%%wdsparql
SELECT ?persona ?personaLabel
WHERE
{
  ?persona wdt:P69 wd:Q5149833 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?institucion ?personaLabel

Esta consulta recupera 250 resultados.

Ahora vamos a ver, de esos 250, cuántos de ellos tienen (o tenían) como ocupación la de escritor de ciencia ficción.

Estudiantes de [Fu Foundation School of Engineering and Applied Science (Q5149833)](https://www.wikidata.org/wiki/Q5149833) cuya ocupación es autor de ciencia ficción:

In [13]:
%%wdsparql
SELECT ?persona ?personaLabel
WHERE
{
  ?persona wdt:P69 wd:Q5149833 ;
           wdt:P106 wd:Q18844224 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?institucion ?personaLabel

,persona,personaLabel
0,http://www.wikidata.org/entity/Q34981,Isaac Asimov


Como vemos, solo Isaac Asimov es devuelto, así que la consulta inicial que solo recuperaba a Asimov como autor de ciencia ficción que había sido educado en Fu Foundation School of Engineering and Applied Science era correcto.

Vamos a hacer lo mismo ahora para las otras dos instituciones restantes que solo devolvían a Asimov como ex-estudiante y autor de ciencia ficción. En caso de que estas dos también recuperen solo a Asimov del total de todos sus estudiantes (antiguos o actuales) podremos concluir que la primera consulta era correcta.

Estudiantes de [Columbia University School of General Studies (Q5149901)](https://www.wikidata.org/wiki/Q5149901):

In [ ]:
%%wdsparql
SELECT ?persona ?personaLabel
WHERE
{
  ?persona wdt:P69 wd:Q5149901 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?institucion ?personaLabel

Esta consulta recupera 69 resultados.

Estudiantes de [Columbia University School of General Studies (Q5149901)](https://www.wikidata.org/wiki/Q5149901) cuya ocupación es autor de ciencia ficción:

In [14]:
%%wdsparql
SELECT ?persona ?personaLabel
WHERE
{
  ?persona wdt:P69 wd:Q5149901 ;
           wdt:P106 wd:Q18844224 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?institucion ?personaLabel

,persona,personaLabel
0,http://www.wikidata.org/entity/Q34981,Isaac Asimov


Estudiantes de [Boys and Girls High School (Q4952707)](https://www.wikidata.org/wiki/Q4952707):

In [ ]:
%%wdsparql
SELECT ?persona ?personaLabel
WHERE
{
  ?persona wdt:P69 wd:Q4952707 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?institucion ?personaLabel

Esta consulta recupera 27 resultados.

Estudiantes de [Boys and Girls High School (Q4952707)](https://www.wikidata.org/wiki/Q4952707) cuya ocupación es autor de ciencia ficción:

In [15]:
%%wdsparql
SELECT ?persona ?personaLabel
WHERE
{
  ?persona wdt:P69 wd:Q4952707 ;
           wdt:P106 wd:Q18844224 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?institucion ?personaLabel

,persona,personaLabel
0,http://www.wikidata.org/entity/Q34981,Isaac Asimov


Como vemos, todas las instituciones devueltas por la consulta original que solo devolvían a Asimov como estudiante con ocupación de escritor de ciencia ficción eran correctas. Esto se debe a que esta profesión es muy concreta, porque es una profesión de un género específico. Además, como hemos podido comprobar, estas instituciones recuperan un número muy pequeño de estudiantes, así que la probabilidad de encontrar otro escritor de ciencia ficción a parte de Asimov se reduce.

Sin embargo, si hacemos una consulta que nos diga el número de estudiantes de la Universidad de Columbia sin centrarnos en la ocupación, nos devuelve 11936 resultados, por lo que evidentemente es mucho más probable que haya más gente con la misma ocupación que Asimov, siendo está la única institución en la que hay más estudiantes con la profesión de autor de ciencia ficción.

### Consulta 11

¿Cuántos asertos hay sobre Asimov en Wikidata? Ten en cuenta que Asimov puede aparece tanto como sujeto como objeto de cada tripleta.

#### Solución:

In [16]:
%%wdsparql
SELECT (COUNT(*) AS ?numeroDeAsertos)
WHERE {
  {
    wd:Q34981 ?relacion ?statement.
  } UNION {
    ?statement ?relacion wd:Q34981.
  }
}

,numeroDeAsertos
0,2277


La idea para la solución ha sido hacer dos consultas "separadas" que son juntadas mediante la operación `UNION`. De esta manera podemos unir las dos consultas en la consulta principal. La primera consulta corresponde a todos los asertos en los que Asimov aparece como sujeto de la tripleta, mientras que en la segunda actúa como objecto de la tripleta. Además de esto, como el enunciado nos pide el número de asertos, necesitamos hacer la función `COUNT` para que nos muestre la suma de los resultados recuperados por las dos consultas previamente unidas.

Dejamos también a continuación el código que nos mostraría el contenido de estos asertos en los que aparece Asimov por si se quisiera comprobar, pero no la ejecutamos porque como bien dice el resultado de la celda de arriba nos devolvería 2277 resultados, haciendo enorme el documento.

In [ ]:
%%wdsparql
SELECT ?statementLabel ?relacionLabel 
WHERE {
  {
    wd:Q34981 ?relacion ?statement.
  } UNION {
    ?statement ?relacion wd:Q34981.
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}

Vamos a hacer también una comprobación de que efectivamente se están sumando bien los resultados devueltos por las dos consultas.

Empecemos por las tripletas que tienen como sujeto a Asimov:

In [17]:
%%wdsparql
SELECT (COUNT(*) AS ?numeroDeAsertos)
WHERE {
  wd:Q34981 ?relacion ?statement.
}

,numeroDeAsertos
0,865


Ahora estudiemos las tripletas que tienen a Asimov como objeto:

In [18]:
%%wdsparql
SELECT (COUNT(*) AS ?numeroDeAsertos)
WHERE {
  ?statement ?relacion wd:Q34981.
}

,numeroDeAsertos
0,1412


Si sumamos los resultados de las dos (865 + 1412) nos da como resultado 2277, que era justamente el resultado devuelto por la primera consulta, así que podemos concluir que su resultado era correcto.

Además, tiene sentido que la consulta que más resultados devuelve sea en la que Asimov es el objeto, ya que en la que él es el sujeto solo está la información que hay en su página de wikidata, pero puede ser objeto de cualquier aserto de cualquier otra página. Por ejemplo, por cada libro suyo se le asigna la relación de autor, pero en su página no aparecen todos sus libros, por lo que solo con este ejemplo vemos que la consulta de Asimov siendo el objeto será mayor que en la que es el sujeto de la tripleta.

**_Fin de la práctica_**